In [2]:
import torch
import import_ipynb
import numpy as np
from models.video_C3D import C3D
import cv2

importing Jupyter notebook from /home/moon/research/sitting-posture-recognition/models/video_C3D.ipynb


In [3]:
video_path = ''
model_path = ''
label_path = ''

In [2]:
torch.backends.cudnn.benchmark = True

def CenterCrop(frame, size):
    h, w = np.shape(frame)[0:2]
    th, tw = size
    x1 = int(round((w - tw) / 2.))
    y1 = int(round((h - th) / 2.))

    frame = frame[y1:y1 + th, x1:x1 + tw, :]
    return np.array(frame).astype(np.uint8)


def center_crop(frame):
    frame = frame[8:120, 30:142, :]
    return np.array(frame).astype(np.uint8)


def main():
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    print("Device being used:", device)

    with open(label_path, 'r') as f:
        class_names = f.readlines()
        f.close()
        
    # init model
    model = C3D(num_classes=6)
    model.load_state_dict(torch.load(model_path, map_location=lambda storage, loc: storage)['state_dict'])
    model.to(device)
    model.eval()

    # read video
    video = video_path

    cap = cv2.VideoCapture(video)

    retaining = True

    clip = []
    while retaining:
        retaining, frame = cap.read()
        print(frame)
        if not retaining and frame is None:
            continue
        tmp_ = cv2.resize(frame, (320, 240))
        tmp = tmp_ - np.array([[[90.0, 98.0, 102.0]]])
        clip.append(tmp)
        if len(clip) == 60:
            inputs = np.array(clip).astype(np.float32)
            inputs = np.expand_dims(inputs, axis=0)
            print(inputs.shape)
            inputs = np.transpose(inputs, (0, 4, 1, 2, 3))
            inputs = torch.from_numpy(inputs)
            inputs = torch.autograd.Variable(inputs, requires_grad=False).to(device)
            with torch.no_grad():
                outputs = model.forward(inputs)

            probs = torch.nn.Softmax(dim=1)(outputs)
            label = torch.max(probs, 1)[1].detach().cpu().numpy()[0]

            # cv2.putText(frame, class_names[label].split(' ')[-1].strip(), (20, 20),
            #             cv2.FONT_HERSHEY_SIMPLEX, 0.6,
            #             (0, 0, 255), 1)
            # cv2.putText(frame, "prob: %.4f" % probs[0][label], (20, 40),
            #             cv2.FONT_HERSHEY_SIMPLEX, 0.6,
            #             (0, 0, 255), 1)
            clip.pop(0)

        cv2.imshow('result', frame)
        cv2.waitKey(30)

    cap.release()
    cv2.destroyAllWindows()

In [2]:
# if __name__ == '__main__':
#     main()